Chama os cachorros

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [2]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

Pré-processamento

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|███████████████████████████████████████| 836/836 [00:04<00:00, 173.00it/s]


REDE CNN

In [34]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

model = Sequential()

### TODO: Define your architecture.

num_classes = 133 #Quantidade de raças de cachorros

#Convolutional layer configuration
input_shape = (224, 224, 3)

#Conv2D(filters, kernel_size, input_shape=input_shape, padding='valid')
#model.add(MaxPooling2D(pool_size, strides))

#model.add(Conv2D(filters=16, kernel_size=(3, 3), input_shape=input_shape, padding='valid', activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))

model.add(BatchNormalization(input_shape=input_shape))

model.add(Conv2D(filters=20, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=20, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=20, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation='relu'))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(GlobalAveragePooling2D())
#model.add(Flatten())
#model.add(Dropout(0.08))

#Dense layer
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.08))

#Dense layer 2
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.08))


#Output layer
model.add(Dense(num_classes, activation='softmax', name='output'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_101 (Bat (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 224, 224, 20)      560       
_________________________________________________________________
batch_normalization_102 (Bat (None, 224, 224, 20)      80        
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 224, 224, 20)      3620      
_________________________________________________________________
batch_normalization_103 (Bat (None, 224, 224, 20)      80        
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 224, 224, 20)      3620      
_________________________________________________________________
batch_normalization_104 (Bat (None, 224, 224, 20)      80        
__________

Compilar modelo

In [35]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=20, batch_size=20, verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20


4120/6680 [=================>............] - ETA: 1128s - loss: 5.0037 - acc: 0.0000e+ - ETA: 638s - loss: 4.9749 - acc: 0.0000e+00 - ETA: 475s - loss: 4.9802 - acc: 0.0000e+0 - ETA: 393s - loss: 4.9910 - acc: 0.0000e+0 - ETA: 344s - loss: 5.0567 - acc: 0.0000e+0 - ETA: 311s - loss: 5.0622 - acc: 0.0000e+0 - ETA: 287s - loss: 5.0444 - acc: 0.0071    - ETA: 269s - loss: 5.0541 - acc: 0.006 - ETA: 255s - loss: 5.0580 - acc: 0.005 - ETA: 244s - loss: 5.0640 - acc: 0.005 - ETA: 235s - loss: 5.0463 - acc: 0.004 - ETA: 227s - loss: 5.0227 - acc: 0.004 - ETA: 220s - loss: 5.0153 - acc: 0.003 - ETA: 214s - loss: 5.0072 - acc: 0.003 - ETA: 209s - loss: 5.0070 - acc: 0.003 - ETA: 204s - loss: 5.0412 - acc: 0.003 - ETA: 200s - loss: 5.0474 - acc: 0.002 - ETA: 197s - loss: 5.0271 - acc: 0.005 - ETA: 193s - loss: 5.0197 - acc: 0.005 - ETA: 190s - loss: 5.0090 - acc: 0.005 - ETA: 188s - loss: 4.9993 - acc: 0.007 - ETA: 185s - loss: 4.9906 - acc: 0.011 - ETA: 183s - loss: 4.9869 - acc: 0.010 - ETA: 1

6680/6680 [==============================] - ETA: 61s - loss: 4.8341 - acc: 0.02 - ETA: 60s - loss: 4.8341 - acc: 0.02 - ETA: 60s - loss: 4.8340 - acc: 0.02 - ETA: 59s - loss: 4.8324 - acc: 0.02 - ETA: 59s - loss: 4.8324 - acc: 0.01 - ETA: 58s - loss: 4.8318 - acc: 0.02 - ETA: 58s - loss: 4.8309 - acc: 0.02 - ETA: 58s - loss: 4.8302 - acc: 0.02 - ETA: 57s - loss: 4.8298 - acc: 0.02 - ETA: 57s - loss: 4.8286 - acc: 0.01 - ETA: 56s - loss: 4.8285 - acc: 0.02 - ETA: 56s - loss: 4.8279 - acc: 0.02 - ETA: 55s - loss: 4.8262 - acc: 0.02 - ETA: 55s - loss: 4.8254 - acc: 0.02 - ETA: 54s - loss: 4.8254 - acc: 0.02 - ETA: 54s - loss: 4.8230 - acc: 0.02 - ETA: 53s - loss: 4.8230 - acc: 0.02 - ETA: 53s - loss: 4.8222 - acc: 0.02 - ETA: 52s - loss: 4.8225 - acc: 0.02 - ETA: 52s - loss: 4.8208 - acc: 0.02 - ETA: 51s - loss: 4.8189 - acc: 0.02 - ETA: 51s - loss: 4.8177 - acc: 0.02 - ETA: 50s - loss: 4.8164 - acc: 0.02 - ETA: 50s - loss: 4.8161 - acc: 0.02 - ETA: 49s - loss: 4.8161 - acc: 0.02 - ETA: 

4160/6680 [=================>............] - ETA: 163s - loss: 4.4131 - acc: 0.050 - ETA: 163s - loss: 4.3582 - acc: 0.075 - ETA: 162s - loss: 4.4625 - acc: 0.050 - ETA: 162s - loss: 4.4917 - acc: 0.037 - ETA: 162s - loss: 4.4952 - acc: 0.030 - ETA: 161s - loss: 4.4583 - acc: 0.025 - ETA: 161s - loss: 4.4636 - acc: 0.028 - ETA: 160s - loss: 4.4895 - acc: 0.031 - ETA: 160s - loss: 4.4709 - acc: 0.033 - ETA: 159s - loss: 4.4680 - acc: 0.035 - ETA: 159s - loss: 4.4680 - acc: 0.031 - ETA: 158s - loss: 4.4656 - acc: 0.029 - ETA: 158s - loss: 4.4534 - acc: 0.034 - ETA: 157s - loss: 4.4453 - acc: 0.035 - ETA: 157s - loss: 4.4675 - acc: 0.033 - ETA: 156s - loss: 4.4849 - acc: 0.031 - ETA: 156s - loss: 4.4818 - acc: 0.029 - ETA: 155s - loss: 4.4824 - acc: 0.030 - ETA: 155s - loss: 4.4942 - acc: 0.031 - ETA: 154s - loss: 4.5051 - acc: 0.032 - ETA: 154s - loss: 4.5106 - acc: 0.031 - ETA: 153s - loss: 4.5121 - acc: 0.031 - ETA: 153s - loss: 4.5129 - acc: 0.032 - ETA: 152s - loss: 4.4994 - acc: 0.0

6680/6680 [==============================] - ETA: 61s - loss: 4.4378 - acc: 0.04 - ETA: 61s - loss: 4.4363 - acc: 0.04 - ETA: 60s - loss: 4.4356 - acc: 0.04 - ETA: 60s - loss: 4.4358 - acc: 0.04 - ETA: 59s - loss: 4.4357 - acc: 0.04 - ETA: 59s - loss: 4.4353 - acc: 0.04 - ETA: 58s - loss: 4.4338 - acc: 0.04 - ETA: 58s - loss: 4.4320 - acc: 0.04 - ETA: 57s - loss: 4.4327 - acc: 0.04 - ETA: 57s - loss: 4.4332 - acc: 0.04 - ETA: 56s - loss: 4.4328 - acc: 0.04 - ETA: 56s - loss: 4.4317 - acc: 0.04 - ETA: 55s - loss: 4.4314 - acc: 0.04 - ETA: 55s - loss: 4.4307 - acc: 0.04 - ETA: 54s - loss: 4.4292 - acc: 0.04 - ETA: 54s - loss: 4.4289 - acc: 0.04 - ETA: 53s - loss: 4.4280 - acc: 0.04 - ETA: 53s - loss: 4.4280 - acc: 0.04 - ETA: 52s - loss: 4.4296 - acc: 0.04 - ETA: 52s - loss: 4.4293 - acc: 0.04 - ETA: 51s - loss: 4.4278 - acc: 0.04 - ETA: 51s - loss: 4.4267 - acc: 0.04 - ETA: 50s - loss: 4.4279 - acc: 0.04 - ETA: 50s - loss: 4.4270 - acc: 0.04 - ETA: 49s - loss: 4.4277 - acc: 0.04 - ETA: 

...

6680/6680 [==============================] - ETA: 62s - loss: 0.9412 - acc: 0.70 - ETA: 61s - loss: 0.9423 - acc: 0.70 - ETA: 61s - loss: 0.9406 - acc: 0.70 - ETA: 60s - loss: 0.9476 - acc: 0.70 - ETA: 60s - loss: 0.9486 - acc: 0.70 - ETA: 59s - loss: 0.9533 - acc: 0.70 - ETA: 59s - loss: 0.9554 - acc: 0.70 - ETA: 58s - loss: 0.9544 - acc: 0.70 - ETA: 58s - loss: 0.9546 - acc: 0.70 - ETA: 57s - loss: 0.9573 - acc: 0.70 - ETA: 57s - loss: 0.9583 - acc: 0.69 - ETA: 56s - loss: 0.9608 - acc: 0.69 - ETA: 56s - loss: 0.9614 - acc: 0.69 - ETA: 55s - loss: 0.9614 - acc: 0.69 - ETA: 55s - loss: 0.9612 - acc: 0.69 - ETA: 54s - loss: 0.9617 - acc: 0.69 - ETA: 54s - loss: 0.9619 - acc: 0.69 - ETA: 53s - loss: 0.9622 - acc: 0.69 - ETA: 53s - loss: 0.9617 - acc: 0.69 - ETA: 52s - loss: 0.9622 - acc: 0.69 - ETA: 52s - loss: 0.9614 - acc: 0.69 - ETA: 51s - loss: 0.9613 - acc: 0.69 - ETA: 51s - loss: 0.9633 - acc: 0.69 - ETA: 50s - loss: 0.9640 - acc: 0.69 - ETA: 50s - loss: 0.9635 - acc: 0.69 - ETA: 

4160/6680 [=================>............] - ETA: 163s - loss: 1.1730 - acc: 0.600 - ETA: 163s - loss: 1.0026 - acc: 0.650 - ETA: 162s - loss: 0.8683 - acc: 0.700 - ETA: 162s - loss: 0.8795 - acc: 0.687 - ETA: 162s - loss: 0.8789 - acc: 0.690 - ETA: 161s - loss: 0.8606 - acc: 0.691 - ETA: 162s - loss: 0.8213 - acc: 0.707 - ETA: 162s - loss: 0.7865 - acc: 0.718 - ETA: 162s - loss: 0.7836 - acc: 0.722 - ETA: 161s - loss: 0.7718 - acc: 0.735 - ETA: 161s - loss: 0.8177 - acc: 0.727 - ETA: 161s - loss: 0.8183 - acc: 0.720 - ETA: 160s - loss: 0.8055 - acc: 0.726 - ETA: 160s - loss: 0.8031 - acc: 0.732 - ETA: 159s - loss: 0.7897 - acc: 0.730 - ETA: 159s - loss: 0.7961 - acc: 0.734 - ETA: 158s - loss: 0.8279 - acc: 0.720 - ETA: 158s - loss: 0.8411 - acc: 0.708 - ETA: 157s - loss: 0.8461 - acc: 0.705 - ETA: 157s - loss: 0.8454 - acc: 0.705 - ETA: 156s - loss: 0.8454 - acc: 0.702 - ETA: 156s - loss: 0.8420 - acc: 0.709 - ETA: 155s - loss: 0.8302 - acc: 0.713 - ETA: 155s - loss: 0.8364 - acc: 0.7

6680/6680 [==============================] - ETA: 62s - loss: 0.8361 - acc: 0.72 - ETA: 61s - loss: 0.8370 - acc: 0.72 - ETA: 61s - loss: 0.8396 - acc: 0.72 - ETA: 60s - loss: 0.8370 - acc: 0.72 - ETA: 60s - loss: 0.8368 - acc: 0.72 - ETA: 59s - loss: 0.8397 - acc: 0.72 - ETA: 59s - loss: 0.8395 - acc: 0.72 - ETA: 58s - loss: 0.8402 - acc: 0.72 - ETA: 58s - loss: 0.8414 - acc: 0.72 - ETA: 57s - loss: 0.8425 - acc: 0.72 - ETA: 57s - loss: 0.8437 - acc: 0.72 - ETA: 56s - loss: 0.8417 - acc: 0.72 - ETA: 56s - loss: 0.8399 - acc: 0.72 - ETA: 55s - loss: 0.8410 - acc: 0.72 - ETA: 55s - loss: 0.8416 - acc: 0.72 - ETA: 54s - loss: 0.8434 - acc: 0.72 - ETA: 54s - loss: 0.8437 - acc: 0.72 - ETA: 53s - loss: 0.8455 - acc: 0.72 - ETA: 53s - loss: 0.8456 - acc: 0.72 - ETA: 52s - loss: 0.8464 - acc: 0.72 - ETA: 52s - loss: 0.8475 - acc: 0.72 - ETA: 51s - loss: 0.8471 - acc: 0.72 - ETA: 51s - loss: 0.8485 - acc: 0.72 - ETA: 50s - loss: 0.8493 - acc: 0.72 - ETA: 50s - loss: 0.8504 - acc: 0.72 - ETA: 

4160/6680 [=================>............] - ETA: 167s - loss: 0.5340 - acc: 0.800 - ETA: 166s - loss: 0.6153 - acc: 0.825 - ETA: 166s - loss: 0.7265 - acc: 0.750 - ETA: 165s - loss: 0.6911 - acc: 0.775 - ETA: 165s - loss: 0.6635 - acc: 0.790 - ETA: 165s - loss: 0.5982 - acc: 0.816 - ETA: 164s - loss: 0.6111 - acc: 0.821 - ETA: 164s - loss: 0.6250 - acc: 0.825 - ETA: 163s - loss: 0.6336 - acc: 0.822 - ETA: 162s - loss: 0.6502 - acc: 0.810 - ETA: 162s - loss: 0.6577 - acc: 0.804 - ETA: 161s - loss: 0.6754 - acc: 0.791 - ETA: 160s - loss: 0.6506 - acc: 0.807 - ETA: 160s - loss: 0.6795 - acc: 0.803 - ETA: 159s - loss: 0.6769 - acc: 0.806 - ETA: 158s - loss: 0.6713 - acc: 0.809 - ETA: 158s - loss: 0.6617 - acc: 0.808 - ETA: 157s - loss: 0.6696 - acc: 0.805 - ETA: 156s - loss: 0.6578 - acc: 0.807 - ETA: 156s - loss: 0.6653 - acc: 0.810 - ETA: 155s - loss: 0.6607 - acc: 0.811 - ETA: 155s - loss: 0.6728 - acc: 0.804 - ETA: 154s - loss: 0.6897 - acc: 0.797 - ETA: 154s - loss: 0.6951 - acc: 0.7

6680/6680 [==============================] - ETA: 62s - loss: 0.7088 - acc: 0.77 - ETA: 61s - loss: 0.7073 - acc: 0.77 - ETA: 61s - loss: 0.7068 - acc: 0.77 - ETA: 60s - loss: 0.7083 - acc: 0.77 - ETA: 60s - loss: 0.7091 - acc: 0.77 - ETA: 59s - loss: 0.7136 - acc: 0.77 - ETA: 59s - loss: 0.7145 - acc: 0.77 - ETA: 58s - loss: 0.7144 - acc: 0.77 - ETA: 58s - loss: 0.7168 - acc: 0.77 - ETA: 57s - loss: 0.7161 - acc: 0.77 - ETA: 57s - loss: 0.7155 - acc: 0.77 - ETA: 56s - loss: 0.7173 - acc: 0.77 - ETA: 56s - loss: 0.7160 - acc: 0.77 - ETA: 55s - loss: 0.7154 - acc: 0.77 - ETA: 55s - loss: 0.7140 - acc: 0.77 - ETA: 54s - loss: 0.7164 - acc: 0.77 - ETA: 54s - loss: 0.7164 - acc: 0.77 - ETA: 53s - loss: 0.7158 - acc: 0.77 - ETA: 53s - loss: 0.7157 - acc: 0.77 - ETA: 52s - loss: 0.7147 - acc: 0.77 - ETA: 52s - loss: 0.7151 - acc: 0.77 - ETA: 51s - loss: 0.7162 - acc: 0.77 - ETA: 51s - loss: 0.7148 - acc: 0.77 - ETA: 50s - loss: 0.7142 - acc: 0.77 - ETA: 50s - loss: 0.7159 - acc: 0.77 - ETA: 

In [37]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 33.4928%
